In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
vocab_size = 1000
# cut texts after this number of words (among top max_features most common words)
sentence_max_lenght = 150


In [ ]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

                                                             
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=sentence_max_lenght, padding='pre')
x_test = sequence.pad_sequences(x_test, maxlen=sentence_max_lenght)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('labels of y_train', set(y_train))

x_train, y_train = torch.LongTensor(x_train).to(device), torch.FloatTensor(y_train).to(device)
x_test, y_test = torch.LongTensor(x_test).to(device), torch.FloatTensor(y_test).to(device)

In [ ]:
class Net4IMDB(torch.nn.Module):
    def __init__(self, vocab_size,
                 embedding_size,
                 hidden_size,
                 output_size):
        super(Net4IMDB, self).__init__()
        # write your model component at here

    def forward(self, x):
        # write model forward at here
        
        return 

In [ ]:
embedding_size = 200
hidden_size = 200
output_size = 1
print('Build model')
model = Net4IMDB(vocab_size, 
                            embedding_size, 
                            hidden_size, 
                            output_size).to(device)
print(model)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=1e-1)

In [ ]:
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [ ]:
epochs = 100
batch_size = 2000
x_train_batch = torch.split(x_train, batch_size, dim=0)
y_train_batch = torch.split(y_train, batch_size, dim=0)
for epoch in range(epochs):
    for i,(x,y) in enumerate(zip(x_train_batch, y_train_batch)):
        y_pred = model.forward(x).squeeze(1)
        loss = criterion(y_pred, y)
        accuracy = binary_accuracy(y_pred, y)
        print('epoch %d | step %d | loss %.4f | accuacy %.2f'%(epoch, i, loss.item(), accuracy.item()), end='\r')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
with torch.no_grad():
    y_pred = model.forward(x_test).squeeze(1)
    binary_accuracy(y_pred, y_test)
    print('accuracy on test: %.4f'%(accuracy))